In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [ ]:
df_encoded = pd.read_csv(r'../cleaned/output_ebay_encoded.csv')
df_dropped = pd.read_csv(r'../cleaned/output_ebay_dropped.csv')

In [ ]:
scaler_x_dropped = StandardScaler()
scaler_y_dropped = StandardScaler()

y_dropped = df_dropped['price_pln'].values
X_dropped = df_dropped.drop(columns=['price_pln'])

X_train_dropped, X_test_dropped, y_train_dropped, y_test_dropped = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

X_train_scaled_dropped = scaler_x_dropped.fit_transform(X_train_dropped)
X_test_scaled_dropped = scaler_x_dropped.transform(X_test_dropped)

y_train_scaled_dropped = scaler_y_dropped.fit_transform(y_train_dropped.reshape(-1, 1)).ravel()
y_test_scaled_dropped = scaler_y_dropped.transform(y_test_dropped.reshape(-1, 1)).ravel()

In [ ]:
scaler_x_encoded = StandardScaler()
scaler_y_encoded = StandardScaler()

y_encoded = df_encoded['price_pln'].values
X_encoded = df_encoded.drop(columns=['price_pln'])

X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

X_train_scaled_encoded = scaler_x_encoded.fit_transform(X_train_encoded)
X_test_scaled_encoded = scaler_x_encoded.transform(X_test_encoded)

y_train_scaled_encoded = scaler_y_encoded.fit_transform(y_train_encoded.reshape(-1, 1)).ravel()
y_test_scaled_encoded = scaler_y_encoded.transform(y_test_encoded.reshape(-1, 1)).ravel()

In [ ]:
param_grid = {
    "C": np.arange(1, 120.0, 1),
    "epsilon": np.arange(0.0, 2.0, 0.1),
    "gamma": np.arange(0.0, 2.0, 0.1),
}

svr_encoded = SVR(kernel='rbf')
grid_search = GridSearchCV(svr_encoded, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled_encoded, y_train_encoded)

print(grid_search.best_estimator_)
best_svr_encoded = grid_search.best_estimator_

In [ ]:
param_grid = {
    "C": np.arange(1, 120.0, 1),
    "epsilon": np.arange(0.0, 2.0, 0.1),
    "gamma": np.arange(0.0, 2.0, 0.1),
}

svr_dropped = SVR(kernel='rbf')
grid_search = GridSearchCV(svr_dropped, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled_dropped, y_train_dropped)

print(grid_search.best_estimator_)
best_svr_dropped = grid_search.best_estimator_

In [ ]:
svr_final_encoded = SVR(kernel='rbf', C=7, epsilon=0.1, gamma=0.1) #Tu paramsy do podmiany

svr_final_encoded.fit(X_train_scaled_encoded, y_train_scaled_encoded)
y_pred_scaled_encoded = svr_final_encoded.predict(X_test_scaled_encoded)
y_pred_encoded = scaler_y_encoded.inverse_transform(y_pred_scaled_encoded.reshape(-1, 1)).ravel()

rmse_encoded = np.sqrt(mean_squared_error(y_test_encoded, y_pred_encoded))
print(f"RMSE dla SVR: {rmse_encoded}")

In [ ]:
svr_final_dropped = SVR(kernel='rbf', C=7, epsilon=0.1, gamma=0.1) #Tu paramsy do podmiany

svr_final_dropped.fit(X_train_scaled_dropped, y_train_scaled_dropped)
y_pred_scaled_dropped = svr_final_dropped.predict(X_test_scaled_dropped)
y_pred_dropped = scaler_y_dropped.inverse_transform(y_pred_scaled_dropped.reshape(-1, 1)).ravel()

rmse_dropped = np.sqrt(mean_squared_error(y_test_dropped, y_pred_dropped))
print(f"RMSE dla SVR: {rmse_dropped}")

In [ ]:
svr_final_encoded.score(X_test_scaled_encoded, y_test_scaled_encoded)
svr_final_dropped.score(X_test_scaled_dropped, y_test_scaled_dropped)